In [ ]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math

In [ ]:
def train_test(epoch, lstm_layer):
    data = pd.read_csv(file_path, sep=';', parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'], index_col='dt')
    data["Global_active_power"] = data["Global_active_power"].fillna(data["Global_active_power"].mean())
    data["Global_reactive_power"] = data["Global_reactive_power"].fillna(data["Global_reactive_power"].mean())
    data["Voltage"] = data["Voltage"].fillna(data["Voltage"].mean())
    data["Global_intensity"] = data["Global_intensity"].fillna(data["Global_intensity"].mean())
    data["Sub_metering_1"] = data["Sub_metering_1"].fillna(data["Sub_metering_1"].mean())
    data["Sub_metering_2"] = data["Sub_metering_2"].fillna(data["Sub_metering_2"].mean())
    data["Sub_metering_3"] = data["Sub_metering_3"].fillna(data["Sub_metering_3"].mean())
    
    data_resample = data.resample('h').mean()
    data_resample.shape

    X = data_resample.dropna().copy()
    y = X['Global_active_power'].shift(-1).ffill()

    X_test = X.tail(720)
    y_test = y.tail(720)

    X.drop(X.tail(720).index, inplace=True)
    y.drop(y.tail(720).index, inplace=True)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.fit_transform(X_val)
    X_test = scaler.fit_transform(X_test)

    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_val = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    model = Sequential()
    model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation="relu"))
    if lstm_layer == 1:
            model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics= [RootMeanSquaredError()])

    history = model.fit(X_train, y_train, epochs=epoch, validation_data=[X_test, y_test], verbose=2, shuffle=False)

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.savefig("80-20_e" +str(epoch) +"_loss_history.jpg")
    plt.show()

    model.save("model_80-20_e" +str(epoch) +".h5")

    predictions = model.predict(X_test)
    X_test_pred = X_test.reshape((X_test.shape[0], 7))
    pred_inv = np.concatenate((predictions, X_test_pred[:, -6:]), axis=1)
    pred_inv = scaler.inverse_transform(pred_inv)
    pred_inv = pred_inv[:,0]

    y_test_inv = y_test.values.reshape((len(y_test), 1))
    inv_y = np.concatenate((y_test_inv, X_test_pred[:, -6:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    aa=[x for x in range(24)]
    plt.plot(aa, inv_y[:24], marker='.', label="actual")
    plt.plot(aa, pred_inv[:24], 'r', marker='.', label="prediction")
    plt.ylabel('Global_active_power', size=15)
    plt.xlabel('Time step', size=15)
    plt.legend(fontsize=15)
    plt.savefig("80-20_e" +str(epoch) +"_predict24.jpg")
    plt.show()

    rmse = np.sqrt(mean_squared_error(inv_y, pred_inv))
    print('Test RMSE: %.3f' % rmse)

In [ ]:
def tscv(epoch, lstm_layer):
    data = pd.read_csv(file_path, sep=';', parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'], index_col='dt')
    data["Global_active_power"] = data["Global_active_power"].fillna(data["Global_active_power"].mean())
    data["Global_reactive_power"] = data["Global_reactive_power"].fillna(data["Global_reactive_power"].mean())
    data["Voltage"] = data["Voltage"].fillna(data["Voltage"].mean())
    data["Global_intensity"] = data["Global_intensity"].fillna(data["Global_intensity"].mean())
    data["Sub_metering_1"] = data["Sub_metering_1"].fillna(data["Sub_metering_1"].mean())
    data["Sub_metering_2"] = data["Sub_metering_2"].fillna(data["Sub_metering_2"].mean())
    data["Sub_metering_3"] = data["Sub_metering_3"].fillna(data["Sub_metering_3"].mean())
    
    data_resample = data.resample('h').mean()
    data_resample.shape

    def CreateLSTM (X_train, y_train, X_test, y_test, x):
        # test pake keras
        model = Sequential()
        model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation="relu"))
        if lstm_layer == 1:
            model.add(LSTM(50, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(50))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam', metrics= [RootMeanSquaredError()])
        history = model.fit(X_train, y_train, epochs=epoch, validation_data=[X_test, y_test], verbose=2, shuffle=False)

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper right')
        plt.savefig("tscv_e" +str(epoch) +"_loss_history " +str(x) +".jpg")
        plt.show()

        return model
    
    def PredictionGAP(X_test, y_test, model, x):
        predictions = model.predict(X_test)
        X_test = X_test.reshape((X_test.shape[0], 7))

        pred_inv = np.concatenate((predictions, X_test[:, -6:]), axis=1)
        pred_inv = scaler.inverse_transform(pred_inv)
        pred_inv = pred_inv[:,0]

        y_test_inv = y_test.values.reshape((len(y_test), 1))
        inv_y = np.concatenate((y_test_inv, X_test[:, -6:]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]

        aa=[x for x in range(24)]
        plt.plot(aa, inv_y[:24], marker='.', label="actual")
        plt.plot(aa, pred_inv[:24], 'r', marker='.', label="prediction")
        plt.ylabel('Global_active_power', size=15)
        plt.xlabel('Time step (Jam)', size=15)
        plt.legend(fontsize=15)
        plt.savefig("tscv_e" +str(epoch) +"_predict24 " +str(x) +".jpg")
        plt.show()

        rmse = np.sqrt(mean_squared_error(inv_y, pred_inv))
        print('RMSE: %.3f' % rmse)  
        
    X = data_resample.dropna().copy()
    y = X['Global_active_power'].shift(-1).ffill()

    X_test = X.tail(720)
    y_test = y.tail(720)

    X.drop(X.tail(720).index, inplace=True)
    y.drop(y.tail(720).index, inplace=True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    X_test = scaler.fit_transform(X_test)
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    tscv = TimeSeriesSplit(n_splits = 4)
    rmse_scores = []
    x = 1

    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        X_train = scaler.fit_transform(X_train)
        X_val = scaler.fit_transform(X_val)

        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))

        print("Iterasi ke-" ,x)
        model = CreateLSTM(X_train, y_train, X_val, y_val, x)

        model.save("model_tscv_e" +str(epoch) +" " +str(x) +".h5")

        predict = PredictionGAP(X_test, y_test, model, x)

        rmse = model.evaluate(X_test, y_test, verbose=0)[1]
        rmse_scores.append(rmse)
        x += 1

    print(rmse_scores)
    global mean_rmse
    mean_rmse = np.mean(rmse_scores)
    global rmse_akhir
    rmse_akhir = rmse_scores[3]
    print("Mean RMSE:", mean_rmse)

In [ ]:
def predict():
    print("Prediction button clicked")

    train_window = tk.Toplevel(root)
    train_window.title("Prediction Window")

    def choose_file_model():
        global file_path_model
        file_path_model = filedialog.askopenfilename(title="Select Model File", filetypes=[("Model Files", "*.h5"), ("All Files", "*.*")])
        if file_path_model:
            print("Model file selected:", file_path_model)

    def choose_file():
        global file_path
        file_path = filedialog.askopenfilename(title="Select File Dataset", filetypes=[("CSV Files", "*.csv"), ("Text Files", "*.txt"), ("All Files", "*.*")])
        if file_path:
            print("File dataset selected:", file_path)

    def predict():
        data = pd.read_csv(file_path, sep=';', parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'], index_col='dt')
        data["Global_active_power"] = data["Global_active_power"].fillna(data["Global_active_power"].mean())
        data["Global_reactive_power"] = data["Global_reactive_power"].fillna(data["Global_reactive_power"].mean())
        data["Voltage"] = data["Voltage"].fillna(data["Voltage"].mean())
        data["Global_intensity"] = data["Global_intensity"].fillna(data["Global_intensity"].mean())
        data["Sub_metering_1"] = data["Sub_metering_1"].fillna(data["Sub_metering_1"].mean())
        data["Sub_metering_2"] = data["Sub_metering_2"].fillna(data["Sub_metering_2"].mean())
        data["Sub_metering_3"] = data["Sub_metering_3"].fillna(data["Sub_metering_3"].mean())
        data_resample = data.resample('h').mean()
        data_resample.shape

        X = data_resample.dropna().copy()
        y = X['Global_active_power'].shift(-1).ffill()

        x_test = X.tail(720)
        y_test = y.tail(720)

        X.drop(X.tail(720).index, inplace=True)
        y.drop(y.tail(720).index, inplace=True)

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.fit_transform(X_val)
        X_test = scaler.fit_transform(X_test)

        X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
        X_val = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
        X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

        model = keras.models.load_model(file_path_model)

        predictions = model.predict(X_test)
        X_test_pred = X_test.reshape((X_test.shape[0], 7))

        pred_inv = np.concatenate((predictions, X_test_pred[:, -6:]), axis=1)
        pred_inv = scaler.inverse_transform(pred_inv)
        pred_inv = pred_inv[:,0]

        y_test_inv = y_test.values.reshape((len(y_test), 1))
        inv_y = np.concatenate((y_test_inv, X_test_pred[:, -6:]), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]

        aa=[x for x in range(24)]
        plt.plot(aa, inv_y[:24], marker='.', label="actual")
        plt.plot(aa, pred_inv[:24], 'r', marker='.', label="prediction")
        plt.ylabel('Global_active_power', size=15)
        plt.xlabel('Time step (Jam)', size=15)
        plt.legend(fontsize=15)
        plt.savefig("predict24.jpg")
        plt.show()

        rmse = np.sqrt(mean_squared_error(inv_y, pred_inv))
        print('Test RMSE: %.3f' % rmse)

        predict_window = tk.Toplevel(train_window)
        predict_window.title("Prediction Result Window")

        prediction_label = tk.Label(predict_window, text="Prediksi model selesai. Hasil Prediksi Bisa Dilihat di Terminal")
        prediction_label.pack(pady=10)

    keterangan_label = tk.Label(train_window, text="Prediksi hanya bisa dilakukan dengan model yang dilatih dengan Split Train-Test 80%-20%")
    keterangan_label.pack(pady=10)

    choose_file_button = tk.Button(train_window, text="Choose File Model", command=choose_file_model)
    choose_file_dataset_button = tk.Button(train_window, text="Choose File Dataset", command=choose_file)
    predict_button = tk.Button(train_window, text="Prediksi", command=predict)

    choose_file_button.pack(pady=10)
    choose_file_dataset_button.pack(pady=10)
    predict_button.pack(pady=10)

In [ ]:
def train():
    # Create a new Toplevel window
    train_window = tk.Toplevel(root)
    train_window.title("Training Options")
    
    def choose_file():
        global file_path
        file_path = filedialog.askopenfilename(title="Select Training File", filetypes=[("CSV Files", "*.csv"), ("Text Files", "*.txt"), ("All Files", "*.*")])
        if file_path:
            print("Training file selected:", file_path)

    def train_model():
        print("Metode train data")
        train_result_window = tk.Toplevel(train_window)
        train_result_window.title("Train Result")

        def update_model_path():
            model_path = "Example_Model_Path"  # You can replace this with the actual path variable
            model_path_label.config(text="Model Path: " + model_path)

        if training_option.get() == "train_test":
            print("train_test")
            # print(epoch_option.get())
            train_test(epoch_option.get(), lstm_option.get())
        elif training_option.get() == "tscv":
            print("TSCV")
            # print(epoch_option.get())
            tscv(epoch_option.get(), lstm_option.get())

        # Create a label to display the model path
        global model_path_label
        model_path_label = tk.Label(train_result_window, text="Training model selesai. Hasil Pelatihan Bisa Dilihat di Terminal")
        model_path_label.pack(pady=10)

        # model_path = "Example_Model_Path"  # You can replace this with the actual path variable
        # model_path_label.config(text="Model Path: " + file_path)

        
        # if training_option.get() == "tscv4" or training_option.get() == "tscv6":
        #     mean_rmse_label = tk.Label(train_result_window, text="Rerata: ")
        #     mean_rmse_label.config(text="Rerata: " +mean_rmse)

        # rmse_akhir_label = tk.Label(train_result_window, text="Model terakhir: ")
        # rmse_akhir_label.config(text="Model terakhir: " +rmse_akhir)

        # Create a button to update the model path
        # update_model_button = tk.Button(train_result_window, text="Update Model Path", command=update_model_path)
        # update_model_button.pack(pady=10)

    def handle_training_selection():
        print("Selected option:", training_option.get())

    global training_option
    training_option = tk.StringVar(value="train_test")
    split_train_radio = tk.Radiobutton(train_window, text="Split Train-Test 80%-20%", variable=training_option, value="train_test", command=handle_training_selection)
    spv_radio = tk.Radiobutton(train_window, text="TSCV", variable=training_option, value="tscv", command=handle_training_selection)

    def handle_epoch_selection():
        print("Selected option:", epoch_option.get())

    global epoch_option
    epoch_option = tk.IntVar(value=4)

    epoch_100 = tk.Radiobutton(train_window, text="Epoch 100", variable=epoch_option, value=4, command=handle_epoch_selection)
    epoch_150 = tk.Radiobutton(train_window, text="Epoch 150", variable=epoch_option, value=150, command=handle_epoch_selection)
    epoch_200 = tk.Radiobutton(train_window, text="Epoch 200", variable=epoch_option, value=200, command=handle_epoch_selection)

    def handle_training_selection():
        print("Selected option:", lstm_option.get())
    
    global lstm_option
    lstm_option = tk.IntVar(value=0)

    lstm2 = tk.Radiobutton(train_window, text="2 layer", variable=lstm_option, value=0, command=handle_epoch_selection)
    lstm3 = tk.Radiobutton(train_window, text="3 layer", variable=lstm_option, value=1, command=handle_epoch_selection)

    choose_file_button = tk.Button(train_window, text="Choose File", command=choose_file)
    train_model_button = tk.Button(train_window, text="Train Model", command=train_model)

    choose_file_button.pack(pady=10)

    mode_training_label = tk.Label(train_window, text="Pilih mode pelatihan model")
    mode_training_label.pack(pady=10)

    split_train_radio.pack()
    spv_radio.pack()

    epoch_training_label = tk.Label(train_window, text="Epoch Training")
    epoch_training_label.pack(pady=10)

    epoch_100.pack()
    epoch_150.pack()
    epoch_200.pack()
    
    mode_lstm_label = tk.Label(train_window, text="Pilih seberapa banyak layer LSTM")
    mode_lstm_label.pack(pady=10)
    lstm2.pack()
    lstm3.pack()

    train_model_button.pack(pady=10)

In [ ]:
root = tk.Tk()
root.title("Prediksi Penggunaan Listrik Pada Sebuah Rumah Menggunakan LSTM")
root.geometry('800x600')
training_button = tk.Button(root, text="Training", command=train)
prediction_button = tk.Button(root, text="Prediction", command=predict)
training_button.pack(pady=10)
prediction_button.pack(pady=10)
root.mainloop()